In [ ]:
from IPython.display import Javascript, display
import jupyanno as ja, numpy as np
zoom_update_js = """
require(['cornerstone-core', 'cornerstoneTools', 'cornerstoneMath'], 
    function(cornerstone, cornerstoneTools, cornerstoneMath){
        var cs_viewer = document.getElementById('{item_name}');
        var viewport = cornerstone.getViewport(cs_viewer);
        viewport.scale += {zoom_factor};
        cornerstone.setViewport(cs_viewer, viewport);
    })
"""
simple_update_js = """
require(['cornerstone-core', 'cornerstoneTools', 'cornerstoneMath'], 
    function(cornerstone, cornerstoneTools, cornerstoneMath){
    var cs_viewer = document.getElementById('{item_name}');
    cornerstoneTools.external.cornerstone = cornerstone
    cornerstoneTools.external.cornerstoneMath = cornerstoneMath
    {simple_numpy_js}
    console.log('Fixing'+cs_viewer+'with'+image);
    var viewport = cornerstone.getDefaultViewportForImage(cs_viewer, image);
    cornerstone.displayImage(cs_viewer, image, viewport);
})
"""
import ipywidgets as ipw
def show_image(obj_id, img_data):
    out_js = ja.utils.fancy_format(
        simple_update_js,
        item_name = obj_id,
        simple_numpy_js = ja.cornerstone.make_pyimage_js(img_data))
    display(Javascript(out_js))

def zoom_viewer(obj_id, zf):
    out_js = ja.utils.fancy_format(
        zoom_update_js,
        item_name = obj_id, zoom_factor=zf)
    display(Javascript(out_js))
    
noisy_img_but = ipw.Button(description='Noisy Image')
noisy_image = np.random.uniform(-1000, 1000, size=(100, 100))
noisy_img_but.on_click(lambda *args: show_image('dicom_viewer', noisy_image))
half_img_but = ipw.Button(description='Half Image')
half_image = np.eye(256)[:128]
half_img_but.on_click(lambda *args: show_image('dicom_viewer', half_image))

zoom_in_but = ipw.Button(description='Zoom In')
zoom_in_but.on_click(lambda *args: zoom_viewer('dicom_viewer', 0.25))
zoom_out_but = ipw.Button(description='Zoom Out')
zoom_out_but.on_click(lambda *args: zoom_viewer('dicom_viewer', -0.25))

In [ ]:
simple_cornerstone_view = """
require.config({
  paths: {
      'cornerstone-core': '//unpkg.com/cornerstone-core@2.2.4/dist/cornerstone.min',
      cornerstoneMath: '//unpkg.com/cornerstone-math@0.1.6/dist/cornerstoneMath.min',
      cornerstoneTools: '//unpkg.com/cornerstone-tools@2.3.9/dist/cornerstoneTools.min'
  },
    shim: {
        'cornerstone-core': {
            exports: 'cornerstone',
            deps: ['jquery']
        }
    }
});
function disableContextMenu(e) {
    $(e).on('contextmenu', function(e) {
        e.preventDefault();
    });
}
require(['cornerstone-core', 'cornerstoneTools', 'cornerstoneMath'], 
    function(cornerstone, cornerstoneTools, cornerstoneMath){
    cornerstoneTools.external.cornerstone = cornerstone
    cornerstoneTools.external.cornerstoneMath = cornerstoneMath
    $("#{item_name}").remove();
    console.log('Loading cs:'+cornerstone+','+cornerstoneMath+','+cornerstoneTools);
    element.append("<div id='{item_name}'></div>");
    
    $("#{item_name}").width("512px");
    $("#{item_name}").height("600px"); 
    var cs_viewer = document.getElementById('{item_name}');
    disableContextMenu(cs_viewer)
    cornerstone.enable(cs_viewer);
    cornerstoneTools.mouseInput.enable(cs_viewer);
    cornerstoneTools.mouseWheelInput.enable(cs_viewer);
    cornerstoneTools.wwwc.activate(cs_viewer, 1); // Left Click
    cornerstoneTools.pan.activate(cs_viewer, 2); // Middle Click
    cornerstoneTools.zoom.activate(cs_viewer, 4); // Right Click
})
"""
Javascript(ja.utils.fancy_format(simple_cornerstone_view, 
                                 item_name='dicom_viewer'))

In [ ]:
ipw.HBox([
    ipw.VBox([noisy_img_but, half_img_but]),
    ipw.VBox([zoom_in_but, zoom_out_but])
])